### Run this in Python kernel

In [1]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

Syntax error compiling at (REPL:0:0).
Unable to resolve symbol: %%html in this context
Syntax error compiling at (REPL:0:0).
Unable to resolve symbol: <style> in this context
Syntax error compiling at (REPL:0:0).
Unable to resolve symbol: body in this context
Syntax error reading source at (REPL:4:17).
Invalid token: font-family:


class clojure.lang.ExceptionInfo: 

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-16" 10000);

### Write a strategy

The following code implements a simple trading strategy:

In a timespan of 10 days (inclusive of today),
- Buy 50 stocks of AAPL on the first day
- Sell 10 stocks of AAPL on every other day

In [4]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 10))                              

(while (pos? @num-of-days)
    (do 
        ;; write your trading strategy here
        (if (= 10 @num-of-days)
            (do
                (order "AAPL" 50) ; buy 50 stocks
                (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get-date)))
            )
        )
        (if (odd? @num-of-days)
            (do
                (order "AAPL" -10) ; sell 10 stocks
                (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get-date)))
            )
        )
        
        ;(update-eval-report (get-date))
        
        ; move on to the next trading day
        (next-date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

; check whether counter == 0
(println ((fn [counter] (str "Counter: " counter)) @num-of-days))

26.093628318359524
Order: 1980-12-17 | AAPL | 50.
Buy 50 stocks of AAPL on 1980-12-16
26.4186732315392
Order: 1980-12-18 | AAPL | -10.
Sell 10 stocks of AAPL on 1980-12-17
27.669667928008312
Order: 1980-12-22 | AAPL | -10.
Sell 10 stocks of AAPL on 1980-12-19
28.803032950478386
Order: 1980-12-24 | AAPL | -10.
Sell 10 stocks of AAPL on 1980-12-23
30.10884791386528
Order: 1980-12-29 | AAPL | -10.
Sell 10 stocks of AAPL on 1980-12-26
29.41870999204002
Order: 1980-12-31 | AAPL | -10.
Sell 10 stocks of AAPL on 1980-12-30
Counter: 0


nil

### New way of using order

In [5]:
(order "AAPL" 10) ;with leverage, exact value trade
(order "AAPL" 10 :leverage false) ;without leverage, exact value trade
(order "AAPL" 10 :remaining true) ;with leverage, remaining value
(order "AAPL" 10 :leverage false :remaining true) ;without leverage, remaining value

29.581621477765204
Order: 1981-01-02 | AAPL | 10.
29.581621477765204
Order: 1981-01-02 | AAPL | 10.
29.581621477765204
Order: 1981-01-02 | AAPL | -10.
29.581621477765204
Order: 1981-01-02 | AAPL | 0.


({:date "1980-12-17", :tic "AAPL", :price 25.9375, :quantity 50, :reference 2} {:date "1980-12-18", :tic "AAPL", :price 26.6875, :quantity -10, :reference 3} {:date "1980-12-22", :tic "AAPL", :price 29.6875, :quantity -10, :reference 5} {:date "1980-12-24", :tic "AAPL", :price 32.5625, :quantity -10, :reference 7} {:date "1980-12-29", :tic "AAPL", :price 36.0625, :quantity -10, :reference 9} {:date "1980-12-31", :tic "AAPL", :price 34.1875, :quantity -10, :reference 11} {:date "1981-01-02", :tic "AAPL", :price 34.625, :quantity 10, :reference 12} {:date "1981-01-02", :tic "AAPL", :price 34.625, :quantity 10, :reference 12} {:date "1981-01-02", :tic "AAPL", :price 34.625, :quantity -10, :reference 12} {:date "1981-01-02", :tic "AAPL", :price 34.625, :quantity 0, :reference 12})

### Calculating indices during the process

#### Moving average

In [6]:
(average (map (fn [_] (Double/parseDouble (get _ :PRC))) (get-prev-n-days :PRC 10 "AAPL"))) ;:PRC should be replaced
                                                                                            ; by the key

Syntax error compiling at (REPL:1:58).
Unable to resolve symbol: get-prev-n-days in this context


class clojure.lang.Compiler$CompilerException: 

##### An example of defining moving average

In [7]:
(defn moving-average
    [key period tic]
    (average (map (fn [_] (Double/parseDouble (get _ key))) (get-prev-n-days key period tic))))

Syntax error (IllegalStateException) compiling def at (REPL:1:1).
moving-average already refers to: #'clojure-backtesting.data/moving-average in namespace: clojure-backtesting.demo


class clojure.lang.Compiler$CompilerException: 

### Check order record

In [8]:
(pprint/print-table (deref order-record))


|      :date | :tic |  :price | :quantity | :reference |
|------------+------+---------+-----------+------------|
| 1980-12-17 | AAPL | 25.9375 |        50 |          2 |
| 1980-12-18 | AAPL | 26.6875 |       -10 |          3 |
| 1980-12-22 | AAPL | 29.6875 |       -10 |          5 |
| 1980-12-24 | AAPL | 32.5625 |       -10 |          7 |
| 1980-12-29 | AAPL | 36.0625 |       -10 |          9 |
| 1980-12-31 | AAPL | 34.1875 |       -10 |         11 |
| 1981-01-02 | AAPL |  34.625 |        10 |         12 |
| 1981-01-02 | AAPL |  34.625 |        10 |         12 |
| 1981-01-02 | AAPL |  34.625 |       -10 |         12 |
| 1981-01-02 | AAPL |  34.625 |         0 |         12 |


nil

### Check portfolio record

In [9]:
;; view final portfolio
(view-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A |     9948 |
|   AAPL | 34.625 | 29.58 |        10 |      295 |


nil

In [10]:
;; view portfolio value and return
(view-portfolio-record)


|      :date | :tot-value | :daily-ret |
|------------+------------+------------|
| 1980-12-16 |     $10000 |      0.00% |
| 1980-12-17 |     $10007 |      0.00% |
| 1980-12-18 |     $10026 |      0.00% |
| 1980-12-22 |     $10096 |      0.01% |
| 1980-12-24 |     $10168 |      0.01% |
| 1980-12-29 |     $10254 |      0.01% |
| 1980-12-31 |     $10295 |      0.00% |
| 1981-01-02 |     $10244 |      0.00% |


nil

### Generate evaluation report

In [11]:
(eval-report)

nil

### Plot variables

In [12]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [13]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

#'clojure-backtesting.demo/data-to-plot

In [14]:
(first data-to-plot)

{:date "1980-12-16", :tot-value 10000, :daily-ret 0.0, :loan 0.0, :leverage 0.0, :plot "portfolio"}

In [15]:
(plot data-to-plot :plot :date :daily-ret)

### Test range of 1 year

In [17]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 250))                              

(while (pos? @num-of-days)
    (do 
        ;; write your trading strategy here
        (if (= 250 @num-of-days)
            (do
                (order "AAPL" 50) ; buy 50 stocks
                (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get-date)))
            )
        )
        (if (= 1 @num-of-days)
            (do
                (order "AAPL" -50) ; sell 10 stocks
                (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get-date)))
            )
        )
        
        (update-eval-report (get-date))
        
        ; move on to the next trading day
        (next-date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

23.990787071305643
Order: 1981-12-29 | AAPL | 50.
Buy 50 stocks of AAPL on 1981-12-28
28.243763401725868
Order: 1982-12-22 | AAPL | -50.
Sell 10 stocks of AAPL on 1982-12-21


nil

In [21]:
(def data (deref portfolio-value))

; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

(plot data-to-plot :plot :date :daily-ret)